## Planning:

Dfs we should have:
* theme_df: theme id; theme name
* df: project_id; project_title; country, region, theme, summary, funding, ...

In [3]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import json


In [1]:
keys = {}

with open("../API_Keys", "r") as file:
    for line in file:
        line = line.strip()
        if line:
            key, value = line.split(" = ")
            keys[key] = value.strip("'")

# Now you can access the values using the key names
charity_key = keys["charity_secret"]


## Creating theme_df

In [ ]:
url = "https://api.globalgiving.org/api/public/projectservice/themes?api_key="

# Send a GET request to the URL
response = requests.get(url+charity_key, headers={"Accept": "application/json"})

# Check if the request was successful (status code 200)
if response.status_code == 200:
    themes_data = response.json()
    themes = themes_data['themes']['theme']

    # Extract IDs and names from themes
    ids = [theme['id'] for theme in themes]
    names = [theme['name'] for theme in themes]

    # Print the IDs and names
    for theme_id, theme_name in zip(ids, names):
        print("ID:", theme_id)
        print("Name:", theme_name)
        print()

else:
    print('Request failed with status code:', response.status_code)


ID: animals
Name: Animal Welfare

ID: children
Name: Child Protection

ID: climate
Name: Climate Action

ID: democ
Name: Peace and Reconciliation

ID: disaster
Name: Disaster Response

ID: ecdev
Name: Economic Growth

ID: edu
Name: Education

ID: env
Name: Ecosystem Restoration

ID: gender
Name: Gender Equality

ID: health
Name: Physical Health

ID: human
Name: Ending Human Trafficking

ID: rights
Name: Justice and Human Rights

ID: sport
Name: Sport

ID: tech
Name: Digital Literacy

ID: hunger
Name: Food Security

ID: art
Name: Arts and Culture

ID: lgbtq
Name: LGBTQIA+ Equality

ID: covid-19
Name: COVID-19

ID: water
Name: Clean Water

ID: disability
Name: Disability Rights

ID: endabuse
Name: Ending Abuse

ID: mentalhealth
Name: Mental Health

ID: justice
Name: Racial Justice

ID: refugee
Name: Refugee Rights

ID: reproductive
Name: Reproductive Health

ID: housing
Name: Safe Housing

ID: agriculture
Name: Sustainable Agriculture

ID: wildlife
Name: Wildlife Conservation



## Regions

In [ ]:
url = "https://api.globalgiving.org/api/public/projectservice/regions?api_key="

# Send a GET request to the URL
response = requests.get(url + charity_key, headers={"Accept": "application/json"})

# Check if the request was successful (status code 200)
if response.status_code == 200:
    themes_data = response.json()
    regions = themes_data['regions']['region']

    # Print regions
    print("Regions:")
    for region in regions:
        print(region['name'])

else:
    print('Request failed with status code:', response.status_code)



Regions:
Africa
Antarctica
Asia and Oceania
Europe and Russia
Middle East
North America
South/Central America and the Caribbean


## Creating df

In [4]:
url = "https://api.globalgiving.org/api/public/projectservice/all/projects/active?api_key="
response = requests.get(url+charity_key, headers={"Accept": "application/json"})

country = 'Spain'
title = ''
region = ''
theme_name = ''

filters = {
    'country': country,
    'title': title,
    'region': region,
    'name': theme_name
}

if response.status_code == 200:
    data = response.json()
    projects = data['projects']['project']

    filtered_projects = []

    for project in projects:
        pass_filters = True

        for filter_column, filter_value in filters.items():
            if filter_column == 'country' and filter_value and project['country'] not in filter_value:
                pass_filters = False
                break
            if filter_column == 'title' and filter_value and project['title'] != filter_value:
                pass_filters = False
                break
            if filter_column == 'region' and filter_value and project['region'] != filter_value:
                pass_filters = False
                break
            if filter_column == 'name' and filter_value:
                themes = project['themes']['theme']
                theme_names = [theme['name'] for theme in themes]
                if filter_value not in theme_names:
                    pass_filters = False
                    break

        if pass_filters:
            filtered_projects.append(project)

    if filtered_projects:
        for project in filtered_projects:
            print("Project Title:", project['title'])
            print("Countries:", project['country'])
            print("Region:", project['region'])
            themes = project['themes']['theme']
            print("Themes:")
            for theme in themes:
                print("\tTheme ID:", theme['id'])
                print("\tTheme Name:", theme['name'])
            print("Summary:", project['summary'])
            print("Funding:", project['funding'])
            print("Goal:", project['goal'])
            donation_options = project['donationOptions']['donationOption']
            print("Donation Options:")
            for donation in donation_options:
                print("\tAmount:", donation['amount'], "$")
                print("\tDescription:", donation['description'])
            print("Project Link:", project['projectLink'])
            print()
    else:
        print("No data found for the specified filters.")

else:
    print('Request failed with status code:', response.status_code)


No data found for the specified filters.


In [5]:
'''stored in a df
url = "https://api.globalgiving.org/api/public/projectservice/all/projects/active?api_key="

# Send a GET request to the URL
response = requests.get(url+charity_key, headers={"Accept":"application/json"})

if response.status_code == 200:
    data = response.json()
    # Extracting the project details
    projects = data['projects']['project']

    # Creating empty lists to store the project details
    project_id = []
    countries = []
    region = []
    theme_id = []
    theme_name = []
    activities = []
    long_term_impact = []
    need = []
    mission = []
    summary = []
    number_of_donations = []
    funding = []
    goal = []
    project_link = []

    # Iterating over each project and extracting the details
    for project in projects:
        project_id.append(project['id'])
        country = project['country']
        countries.append(country)
        region.append(project['region'])
        themes = project['themes']['theme']
        theme_id.append([theme['id'] for theme in themes])
        theme_name.append([theme['name'] for theme in themes])
        activities.append(project['activities'])
        long_term_impact.append(project['longTermImpact'])
        need.append(project['need'])
        mission.append(project['organization']['mission'])
        summary.append(project['summary'])
        number_of_donations.append(project['numberOfDonations'])
        funding.append(project['funding'])
        goal.append(project['goal'])
        project_link.append(project['projectLink'])

    # Creating a DataFrame from the extracted project details
    df = pd.DataFrame({
        'project id': project_id,
        'countries': countries,
        'region': region,
        'theme': theme_id,
        'theme name': theme_name,
        'activities': activities,
        'long term impact': long_term_impact,
        'need': need,
        'mission': mission,
        'summary': summary,
        'number of donations': number_of_donations,
        'funding': funding,
        'goal': goal,
        'project link': project_link
    })

    # Printing the DataFrame
    print(df)
else:
    print('Request not successful')

'''

'stored in a df\nurl = "https://api.globalgiving.org/api/public/projectservice/all/projects/active?api_key="\n\n# Send a GET request to the URL\nresponse = requests.get(url+charity_key, headers={"Accept":"application/json"})\n\nif response.status_code == 200:\n    data = response.json()\n    # Extracting the project details\n    projects = data[\'projects\'][\'project\']\n\n    # Creating empty lists to store the project details\n    project_id = []\n    countries = []\n    region = []\n    theme_id = []\n    theme_name = []\n    activities = []\n    long_term_impact = []\n    need = []\n    mission = []\n    summary = []\n    number_of_donations = []\n    funding = []\n    goal = []\n    project_link = []\n\n    # Iterating over each project and extracting the details\n    for project in projects:\n        project_id.append(project[\'id\'])\n        country = project[\'country\']\n        countries.append(country)\n        region.append(project[\'region\'])\n        themes = project[\